# 3.5 Streams

In [25]:
; define as macros to override Schemes eager evalutation
(define-syntax delay
               [(delay ?exp) (memo-proc (lambda() ?exp))])
(define-syntax cons-stream
               [(cons-stream ?a ?b) (cons ?a (delay ?b))])

(define the-empty-stream '())

(define (stream-null? s)
  (eq? s the-empty-stream))
(define (stream-car stream)
  (car stream))
(define (stream-cdr stream)
  (force (cdr stream)))

(define (stream-ref s n)
  (if (= n 0)
      (stream-car s)
      (stream-ref (stream-cdr s) (- n 1))))

(define (stream-map proc s)
  (if (stream-null? s)
      the-empty-stream
      (cons-stram
       (proc (stream-car s))
       (stream-map proc (stream-cdr s)))))

(define (stream-for-each proc s)
  (if (stream-null? s)
      'done
      (begin
       (proc (stream-car s))
       (stream-for-each proc
                        (stream-cdr s)))))

(define (stream-filter pred stream)
  (cond ((stream-null? stream)
         the-empty-stream)
        ((pred (stream-car stream))
         (cons-stream
          (stream-car stream)
          (stream-filter
           pred
           (stream-cdr stream))))
        (else (stream-filter
               pred
               (stream-cdr stream)))))

(define (stream-enumerate-interval low high)
  (if (> low high)
      the-empty-stream
      (cons-stream
       low
       (stream-enumerate-interval (+ low 1)
                                  high))))

(define (force delayed-object)
  (delayed-object))

(define (memo-proc proc)
  (let ((already-run? #f)
        (result #f))
    (lambda ()
      (if (not already-run?)
          (begin (set! result (proc))
                 (set! already-run? #t)
                 result)
          result))))

(define (display-stream s)
  (stream-for-each display-line s))

(define (display-line x)
  (newline)
  (display x))

## 3.50

In [26]:
; take proc of n args along with n streams
; apply proc to to all elements of each stream
(define (stream-map proc . argstreams)
  (if (stream-null? (car argstreams))
      the-empty-stream
      (cons-stream
       (apply proc (map stream-car argstreams))
       (apply stream-map
              (cons proc
                    (map stream-cdr
                         argstreams))))))

## 3.51

In [27]:
; prints and returns argument
(define (show x)
  (display-line x)
  x)

(define x
  (stream-map
   show
   (stream-enumerate-interval 0 10)))


0

In [28]:
(stream-ref x 5)


1
2
3
4
5

5

In [29]:
(stream-ref x 7)


6
7

7

## 3.52

In [34]:
(define sum 0)

sum

0

In [35]:
(define (accum x)
  (set! sum (+ x sum))
  sum)

sum

0

In [36]:
(define seq
  (stream-map
   accum
   (stream-enumerate-interval 1 20)))

sum

1

In [37]:
(define y (stream-filter even? seq))

sum

6

In [38]:
(define z
  (stream-filter
   (lambda (x)
     (= (remainder x 5) 0)) seq))

sum

10

In [39]:
(stream-ref y 7)

136

In [40]:
sum

136

In [41]:
(display-stream z)


10
15
45
55
105
120
190
210

done

In [42]:
sum

210

The results above would be different if `delay` did not use memoization through `memo-proc`. This is because `sum` is mutated each time `seq` is evaluated.